In [1]:
import pandas as pd
import random
import sympy as sym
import numpy as np

pd.set_option('display.max_rows', None)

In [66]:
def neohooke_uniaxial(c10,stretch):
    #returns cauchy stress at yy direction
    C11 = sym.Symbol('C11')
    C12 = sym.Symbol('C12')
    C13 = sym.Symbol('C13')
    C21 = sym.Symbol('C21')
    C22 = sym.Symbol('C22')
    C23 = sym.Symbol('C23')
    C31 = sym.Symbol('C31')
    C32 = sym.Symbol('C32')
    C33 = sym.Symbol('C33')
    C = sym.Matrix([[C11,C12,C13], [C21,C22,C23], [C31,C32,C33]]) 
    i1=sym.trace(C)
    #Generate SEF
    #symbolic C10
    c10s = sym.Symbol('c10s')
    sef=c10s*(i1-3)
    #Second Piola Kirchoff Stresses
    S11=2*sym.diff(sef,C11)
    S12=2*sym.diff(sef,C12)
    S13=2*sym.diff(sef,C13)
    S21=2*sym.diff(sef,C21)
    S22=2*sym.diff(sef,C22)
    S23=2*sym.diff(sef,C23)
    S31=2*sym.diff(sef,C31)
    S32=2*sym.diff(sef,C32)
    S33=2*sym.diff(sef,C33)
    #
    S = sym.Matrix([[S11,S12,S13], [S21,S22,S23], [S31,S32,S33]])
    #Deformation Gradient
    F=sym.Matrix([[1/(np.sqrt(stretch)),0,0], [0,stretch,0], [0,0,1/(np.sqrt(stretch))]]) 
    Ft=sym.transpose(F)
    C=Ft*F
    Jac=sym.det(C)
    C11v=C[0,0]
    C12v=C[0,1]
    C13v=C[0,2]
    C21v=C[1,0]
    C22v=C[1,1]
    C23v=C[1,2]
    C31v=C[2,0]
    C32v=C[2,1]
    C33v=C[2,2]
    
    cauchy=(1/Jac)*(F*S*Ft)
    stress=cauchy[1,1]-cauchy[0,0]
    stress_abq=sym.Matrix([[0,0,0], [0, stress,0], [0,0,0]]) 
    s22_val=stress_abq.subs([(C11, C11v), (C12, C12v), 
                                  (C13, C13v),(C21, C21v), 
                                  (C22, C22v), (C23, C23v),
                                  (C31, C31v), (C32, C32v), 
                                  (C33, C33v),(c10s,c10)])
    
    return stretch,s22_val[1,1]

def get_curve(c10,stretch_min,stretch_max,ninc):
    #stores neohooke uniaxial runs between minimum and a maximum stretch
    lst=[neohooke_uniaxial(c10,stretch) for stretch in np.linspace(stretch_min,stretch_max,ninc)]
    return lst


In [67]:
#Initial data-----------------------------------------------------------------------------------------------------

C10min=0.1     #min C10 value
C10max=10.0   #max C10 value
decimals=2   #number of decimal cases of C10 variable
n=10               #number of C10 variables
#
st_max=3.0             #applied stretch
st_min=1.0
ninc=100        #number of stretch increments
#------------------------------------------------------------------------------------------------------------------

In [69]:
#add random c10s to dataframe
df = pd.DataFrame(np.round(np.random.uniform(C10min, C10max, size=(n, 1)), decimals),columns=['c10'])
#remove duplicates and complete list
df=df.drop_duplicates()
missing=n-df.shape[0]

while missing!=0:
    new = pd.DataFrame(np.round(np.random.uniform(C10min, C10max, size=(missing, 1)), decimals),columns=['C10'])
    df=pd.concat([df,new], axis=0)
    df=df.reset_index(drop=True)
    df=df.drop_duplicates()
    missing=n-C10df.shape[0]

In [70]:
#generate (x,y) data for each unique c10 at the dataframe
df['data']=df.apply(lambda x : get_curve(x,st_min,st_max,ninc), axis=1)
df

,c10,data
0,4.45,"[(1.0, 0), (1.02020202020202, 0.53946586577849..."
1,4.61,"[(1.0, 0), (1.02020202020202, 0.55886239128963..."
2,1.77,"[(1.0, 0), (1.02020202020202, 0.21457406346695..."
3,8.55,"[(1.0, 0), (1.02020202020202, 1.03650183200138..."
4,6.67,"[(1.0, 0), (1.02020202020202, 0.80859265724551..."
5,2.92,"[(1.0, 0), (1.02020202020202, 0.35398659057824..."
6,0.76,"[(1.0, 0), (1.02020202020202, 0.09213349617790..."
7,7.90,"[(1.0, 0), (1.02020202020202, 0.95770344711238..."
8,7.57,"[(1.0, 0), (1.02020202020202, 0.91769811324566..."
9,2.12,"[(1.0, 0), (1.02020202020202, 0.25700396302256..."


In [5]:
df.to_pickle('./data.pkl')